In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from DBIO import YouBikeDataManager

db_path = '../youbike_data.db'
manager = YouBikeDataManager(db_path)
sno = "500101024"
data = manager.load_and_preprocess_station(sno)

In [3]:
data.columns

Index(['available_rent_bikes', 'available_return_bikes', 'capacity', 'demand'], dtype='object')

In [4]:
data.head()

,available_rent_bikes,available_return_bikes,capacity,demand
mday,,,,
2024-11-18 12:00:00,44,6,50,0
2024-11-18 13:00:00,48,2,50,-4
2024-11-18 14:00:00,45,5,50,3
2024-11-18 15:00:00,40,10,50,5
2024-11-18 16:00:00,30,20,50,10


In [5]:
# Expected frequency (e.g., hourly data)
full_index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')
missing_timestamps = full_index.difference(data.index)

print(f"Missing timestamps:\n{missing_timestamps}")

Missing timestamps:
DatetimeIndex([], dtype='datetime64[ns]', freq='h')


/tmp/ipykernel_14669/3433340772.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')


In [6]:
data = data.reindex(full_index)


In [7]:
data['available_rent_bikes'] = data['available_rent_bikes'].interpolate(method='linear')
data['available_return_bikes'] = data['available_return_bikes'].interpolate(method='linear')
data['demand'] = data['demand'].interpolate(method='linear')

In [8]:
data['is_missing'] = data['demand'].isnull().astype(int)

In [9]:
hourly_avg = data.groupby(data.index.hour).mean()
data = data.fillna(hourly_avg)
